In [1]:
import os
project_root = os.pardir
import sys
src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)
import numpy as np

import general as gen
import matplotlib.pyplot as plt
from visualisation.visualise import *
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
input_dir = os.path.join(project_root, 'data', 'processed')
input_filenames = [
                    #single years, deaths all
                    ['mcres_flows1996-1997_deathsall.pkl',
                        'res_flows1996-1997_deathsall.pkl'],
                    ['mcres_flows1996-1997_deaths1996-2000.pkl',
                        'res_flows1996-1997_deaths1996-2000.pkl']
                ]

In [ ]:
#make filepaths
input_filepaths = []
for filename_pair in input_filenames:
    mc_filepath = os.path.join(input_dir, filename_pair[0])
    actual_filepath = os.path.join(input_dir, filename_pair[1])
    filepath_pair = (mc_filepath, actual_filepath)
    input_filepaths += [filepath_pair]

In [ ]:
def RatiosOverlay(
                    data_list, ell_values,
                    fig_dir=None, shift=False, cal_at=None
                ):
    '''
        Plots of ratio against ell value. Overlay plots for different death years.

    Specifically, overlay each set of data in the lists [mc_res] and
    [actual_res] and rescale so that they are overlayed and the shape can be
    compared. Rescaling is done by shifting a curve up the y-axis *after* logs
    have been taken of the ratios.

    Intended to determine whether the behaviour for high ell values looks the
    same no matter how long we wait for deaths.

    Args:
        - data_list: list of tuples of filepaths in format
        [(mc_res_1, actual_res_1), ..., (mc_res_n, actual_res_n)]. Filepaths
        are for pickle files that contain the dictionaries with keys as ell
        values such that:
            - mc_res_i[ell] = [n_1, n_2, ..., n_i, ...] where n_i is the
            number of pairs of dead nodes between which the shortest path is
            ell in the ith Monte Carlo run,
            - actual_res_i[ell]= 'number of pairs of dead nodes between which
            the shortest path is ell in actual LFN'
        - ell_values: list of ell values for which we want to generate box plots
        - fig_dir: directory where figures are stored
        - shift: bool saying whether we will calibrate
        - cal_at: int saying which ell_value we will calibrate at
    Returns:
        - plot
    '''
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1) #first row, first column, one subplot
    first_loop = True
    for filepath_pair in data_list:
        mc_filepath = filepath_pair[0]
        actual_filepath = filepath_pair[1]
        #check filepaths corresponds to same flow years and death years
        gen.CheckSameData(mc_filepath, actual_filepath)
        #load dicts from pickle files
        mc_res = gen.GetPkl(mc_filepath)
        actual_res = gen.GetPkl(actual_filepath)
        #get ratios
        ratios = GetRatios(mc_res, actual_res)
        log_ratios = dict()
        for key in ratios.keys():
            log_ratios[key] = np.log(ratios[key])
        #set shift calibrator (or if it already exists do shift)
        if type(cal_at)==int: #check that we want calibration
            if first_loop:
                #get the ratio value against which we will cal the others
                cal_value = log_ratios[cal_at]
                cal_log_ratios = log_ratios
            else:
                #calibrate the ratios against the first data set
                cal_difference = log_ratios[cal_at] - cal_value
                cal_log_ratios = dict()
                for key in log_ratios.keys():
                    cal_log_ratios[key] = log_ratios[key] - cal_difference
                assert np.isclose(
                                    cal_log_ratios[cal_at],
                                    cal_value,
                                    rtol=1e-05, atol=1e-08
                                    )
            x = cal_log_ratios.keys()
            y = cal_log_ratios.values()
        else:
            x = log_ratios.keys()
            y = log_ratios.values()
        #add line plot
        start = min(ell_values)
        end = max(ell_values)
        info = mc_filepath
        label = 'Flows: ' + gen.GetFlowYears(info) \
                + ',  Deaths: ' \
                + gen.GetDeathYears(info) \
                + ' ' \
                + gen.GetNrand(info)
        ax.plot(
                x[start:end],
                y[start:end],
                label=label
                )
        #add titles, axis labels, and formatting
        plt.legend()
        plt.title('Line chart showing ratios for different numbers of ' \
                  'death years. With calibration at ' + str(cal_at))
        first_loop=False
RatiosOverlay(
                data_list=input_filepaths, ell_values=range(1, 8),
                fig_dir=None, shift=True, cal_at=4
                )

In [6]:
#if use 'output_dir_test' get error, if use another variable name then no error
output_dir = 'hello'
print(output_dir)
os.path.join(ouput_dir, 'x')

hello


NameError: name 'ouput_dir' is not defined